In [1]:
#data at https://moneypuck.com/data.htm
import pandas as pd
import os
import json

In [6]:
folder_path = '/Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'jsons')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

        # Convert to list-of-dict format
        data_as_json = df.to_dict(orient='records')

        # Define output filename (.txt with same name as .csv)
        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")

        # Save JSON to .txt
        with open(output_file, 'w') as f:
            json.dump(data_as_json, f, indent=4)

        print(f"Saved {output_file}")

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies.txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies_(4).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(15).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/teams_(2).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies_(14).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(13).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/da

In [4]:
folder_path = '/Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'texts')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert each row into a readable paragraph
def row_to_text(row):
    parts = [f"{col}: {row[col]}" for col in df.columns]
    return ", ".join(parts)

# # # # Combine all rows into one big string
# full_text = '\n\n'.join(df.apply(row_to_text, axis=1))

# # Save to text file
# with open('nhl_stats_full.txt', 'w') as f:
#     f.write(full_text)

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

        # Convert to list-of-dict format
        full_text = '\n\n'.join(df.apply(row_to_text, axis=1))

        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")
        with open(output_file, 'w') as f:
            f.write(full_text)

        print(f"Saved {output_file}")


Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/skaters_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/goalies.txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/goalies_(4).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/lines_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/skaters_(15).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/skaters_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/teams_(2).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/goalies_(14).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/lines_(13).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts/lines_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/da

In [ ]:
folder_path = '/Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'texts_chunked')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

MAX_CHARS = 1000  # Max characters per chunk

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

        # Convert a row into a readable format with ability to slice
        def row_to_text_segments(row, max_len):
            name = f"name: {row['name']}" if 'name' in row else ""
            season = f"season: {row['season']}" if 'season' in row else ""
            context = f"{name}, {season}"
            other_parts = [f"{col}: {row[col]}" for col in df.columns if col not in ['name', 'season']]
            full_row = context + ", " + ", ".join(other_parts)

            # Break into multiple parts if necessary
            if len(full_row) <= max_len:
                return [full_row]
            else:
                segments = []
                current = context + ", "
                for part in other_parts:
                    if len(current) + len(part) + 2 > max_len:  # +2 for ", "
                        segments.append(current.rstrip(", "))
                        current = context + ", " + part + ", "
                    else:
                        current += part + ", "
                if current:
                    segments.append(current.rstrip(", "))
                return segments

        chunks = []
        current_chunk = ""

        for _, row in df.iterrows():
            segments = row_to_text_segments(row, MAX_CHARS)

            for segment in segments:
                segment += "\n\n"
                if len(current_chunk) + len(segment) <= MAX_CHARS:
                    current_chunk += segment
                else:
                    chunks.append(current_chunk.strip())
                    current_chunk = segment
        if current_chunk:
            chunks.append(current_chunk.strip())

        # Join and save
        final_text = '\n\n'.join(chunks)
        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")

        with open(output_file, 'w') as f:
            f.write(final_text)

        print(f"Saved {output_file} with {len(chunks)} chunks.")


Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/skaters_(9).txt with 22175 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/goalies.txt with 445 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/goalies_(4).txt with 410 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/lines_(5).txt with 50304 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/skaters_(15).txt with 23775 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/skaters_(5).txt with 20975 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/teams_(2).txt with 600 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/texts_chunked/goalies_(14).txt with 535 chunks.
Saved /Users/carlg/Desktop/Code/School/Columbi